In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math
# from scipy.spatial import KDTree
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from multiprocessing import Pool

from itertools import product

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

# disable chain assignment warnings
pd.options.mode.chained_assignment = None

In [2]:
def extend(submission, hits, limit=0.04, num_neighbours=18, threshold=0.01):

    df = submission.merge(hits,  on=['hit_id'], how='left')
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-90,90,1):

        df1 = df.loc[(df.arctan2>(angle-1.5)/180*np.pi) & (df.arctan2<(angle+1.5)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<3: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1[['x', 'y', 'z']].values.T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        c = np.cos(a)
        s = np.sin(a)

        tree = KDTree(np.column_stack([c, s, r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]
            c0 = c[idx0]
            c1 = c[idx1]
            s0 = s[idx0]
            s1 = s[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0)

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1)

            ## extend start point
            ns = tree.query([[c0, s0, r0]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r0 - r[ns], a0 - a[ns])
            diff = 1 - np.cos(direction - direction0)
            ns = ns[(r0 - r[ns] > threshold) & (diff < (1 - np.cos(limit)))]
            for n in ns: df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

            ## extend end point
            ns = tree.query([[c1, s1, r1]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns] - r1, a[ns] - a1)
            diff = 1 - np.cos(direction - direction1)
            ns = ns[(r[ns] - r1 > threshold) & (diff < (1 - np.cos(limit)))]
            for n in ns:  df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [3]:
# Change this according to your directory preferred setting
path_to_train = "./data/train_1"

# This event is in Train_1
event_prefix = "event000001000"

In [4]:
weights_0 = np.array([1.1, 1.1, .375, .25, 0.05, 0.05, 0])
weights_2 = np.array([1.05, 1.05, .37, .25, .01, .045, 0])
weights_3 = np.array([1.08, 1.08, .38,.25, 0.009, 0.0001, 0])
weights_4 = np.array([1.1, 1.1, .5, .25, 0.008, 0.001, 0])
weights_5 = np.array([1.01, 1.01, .4048, .2, 2e-16, 2e-16, 0])
weights_9 = np.array([1.02, 1.02, .38, .25, 0.009, 0.02, 0])

weights_1 = np.array([1.08, 1.08, .38,.25, 0.0001, 0.0001, 0])
weights_6 = np.array([1.08, 1.08, .38, .25, 2e-4, 0, 0])
weights_8 = np.array([1.07, 1.07, .37, .25, 0.0002, 0.0001, 0])
weights_12 = np.array([1.01, 1.01, .42, .25, 0.0023, 0.001, 0])
weights_13 = np.array([1.0, 1.0, .40, .20, 0.0023, 0.0001, 0])
weights_14 = np.array([1.02, 1.02, .40, .24, 0.0002, 0, 0])
weights_7 = np.array([1.08, 1.08, .40, .20, 0.0023, 0, 0])

weights_10 = np.array([1.02, 1.02, .42, .24, 0.0002, 0, 0])
weights_11 = np.array([1.08, 1.08, .40, .20, 0.0023, 0.0001, 0])
weights_15 = np.array([1.01, 1.01, .42, .25, 0.0002, 0, 0])
weights_16 = np.array([1.005, 1.005, .40, .25, 0.0002, 0, 0])
weights_17 = np.array([1.02, 1.02, .42, .24, 0.02, 0, 0])
weights_18 = np.array([1.033, 1.033, .376, .2, 0.0002, 0, 0])
weights_19 = np.array([0.9734, 0.9734, .4072, .2217, 0.0, 0.0, 0])
weights_20 = np.array([1.0, 1.0, .42, .24, 0.0, 0, 0])
weights_21 = np.array([1.02, 1.02, .42, .24, 0.0, 0, 0])
weights_22 = np.array([1.05, 1.05, .4, .25, 0.0, 0, 0])
weights_23 = np.array([1.05, 1.05, .5, .25, 0.0, 0, 0])
weights_24 = np.array([1.05, 1.05, .4, .20, 0.0, 0, 0])
weights_26 = np.array([1.2, 1.2, .45, .15, 0.0, 0, 0])
weights_27 = np.array([1.2, 1.2, .45, .20, 0.0, 0, 0])
weights_28 = np.array([1.2, 1.2, .45, .25, 0.0, 0, 0])
weights_29 = np.array([1.09, 1.09, .5, .20, 0.0, 0, 0])
weights_25 = np.array([1.09, 1.09, .45, .20, 0.0, 0, 0])
weights_30 = np.array([0.9, .9, .35, .15, 0.0, 0.007, 0.007])
weights_31 = np.array([1.09, 1.09, .45, .20, 0.0, 0.005, 0.005])
weights_34 = np.array([1.09, 1.09, .45, .20, 0.0, 0.01, 0.01])
weights_36 = np.array([1.0, 1.0, .35, .2, 0.0, 0.02, 0.02])
weights_32 = np.array([0.9, .9, .35, .2, 0.0, 0.01, 0.01])
weights_33 = np.array([1.0, 1.0, .35, .2, 0.0, 0.01, 0.01])
weights_37 = np.array([0.9, .9, .4, .2, 0.0, 0.015, 0.015]) # raise w4
weights_38 = np.array([0.9, .9, .4, .25, 0.0, 0.01, 0.01])  # raise w3
weights_39 = np.array([0.9, .9, .45, .2, 0.0, 0.01, 0.01])
weights_40 = np.array([0.95, .95, .4, .2, 0.0, 0.01, 0.01])
weights_41 = np.array([0.95, .95, .45, .2, 0.0, 0.01, 0.01])  # 38 + 40
weights_42 = np.array([0.95, .95, .4, .2, 0.0, 0.008, 0.008]) # w4 lowered
weights_43 = np.array([1.0, 1.0, .4, .2, 0.0, 0.01, 0.01])    # w1 raised
weights_44 = np.array([0.95, .95, .4, .2, 0.0, 0.012, 0.012]) # w4 raised
weights_45 = np.array([1.0, 1.0, .4, .2, 0.0, 0.01, 0.01])  # raise w1 from 35
weights_46 = np.array([0.9, .9, .4, .2, 0.0, 0.009, 0.009]) # lower w4
weights_47 = np.array([0.85, .85, .4, .2, 0.0, 0.01, 0.01]) # lower w1
weights_49 = np.array([0.9, .9, .4, .2, 0.0, 0.007, 0.007]) # still lower w4
weights_48 = np.array([0.9, .9, .4, .2, 0.0, 0.008, 0.008]) # lower w4
weights_51 = np.array([0.9, .9, .4, .2, 0.0, 0.009, 0.009]) # lower w4
weights_50 = np.array([0.9, .9, .4, .2, 0.0, 0.011, 0.011]) # higher w4
weights_52 = np.array([0.9, .9, .4, .2, 0.0, 0.012, 0.012]) # higher w4
weights_53 = np.array([0.9, .9, .4, .2, 0.0, 0.013, 0.013]) # higher w4
weights_54 = np.array([0.9, .9, .4, .2, 0.0, 0.014, 0.014]) # higher w4
weights_55 = np.array([0.9, .9, .4, .2, 0.0, 0.015, 0.015]) # higher w4
weights_56 = np.array([0.9, .9, .5, .2, 0.0, 0.01, 0.01])     # raise w2
weights_60 = np.array([0.7, .7, .5, .13, 0.0, 0.01, 0.01])     # raise w2, lower w1, w3
weights_59 = np.array([0.9, .9, .55, .1, 0.0, 0.01, 0.01])     # raise w2, lower w3
weights_35 = np.array([0.9, .9, .4, .2, 0.0, 0.01, 0.01])     
weights_57 = np.array([0.9, .9, .5, .15, 0.0, 0.01, 0.01]) # second best
weights_61 = np.array([0.9, .9, .4, .1, 0.0, 0.01, 0.01])  # lower w3 some more
weights_62 = np.array([0.9, .9, .45, .1, 0.0, 0.01, 0.01])  # lower w3 and raise w2
weights_63 = np.array([0.9, .9, .5, .1, 0.0, 0.01, 0.01])  # raise w2 some more
weights_64 = np.array([0.9, .9, .5, .07, 0.0, 0.01, 0.01])  # raise w2 and lower w3 some more
weights_65 = np.array([0.9, .9, .35, .15, 0.0, 0.01, 0.01])   # lower w2 a bit
weights_66 = np.array([0.9, .9, .35, .2, 0.0, 0.01, 0.01])   # lower w2 a bit and raise w3
weights_67 = np.array([0.95, .95, .3, .2, 0.0, 0.012, 0.012])# weights from r optimization
weights_68 = np.array([0.96, .96, .35, .2, 0.0, 0.0085, 0.0085])# weights from r optimization
weights_69 = np.array([0.9, .9, .4, .15, 0.0, 0.0085, 0.0085])   # w4 down
weights_70 = np.array([1.0, 1.0, .4, .1214, 0.0, 0.008, 0.008])   # w4 down
weights_71 = np.array([0.9, 0.9, .5, .15, 0.0, 0.0085, 0.0085])   # w4 down
weights_73 = np.array([0.9, .9, .45, .15, 0.0, 0.01, 0.01])   # w2 up a bit
weights_74 = np.array([0.9, .9, .35, .2, 0.0, 0.011, 0.011])   # w2 down and w3 up
weights_72 = np.array([0.9, .9, .4, .15, 0.0, 0.012, 0.012])   # w4 up a bit
weights_75 = np.array([0.9, .9, .4, .15, 0.0, 0.014, 0.014]) 
weights_76 = np.array([0.9, .9, .4, .20, 0.0, 0.014, 0.014]) 
weights_77 = np.array([0.85, 0.85, .4, .20, 0.0, 0.012, 0.012]) 
weights_78 = np.array([0.9, .9, .4, .15, 0.005, 0.01, 0.01]) 
weights_79 = np.array([0.9, .9, .4, .15, 0.0005, 0.01, 0.01]) 
weights_80 = np.array([0.9, .9, .4, .15, 0.01, 0.01, 0.01]) 
weights_82 = np.array([0.96, .96, .45, .126, 0.0, 0.007, 0.007])  
weights_84 = np.array([0.96, .96, .45, .15, 0.0, 0.0108, 0.0108])  
weights_86 = np.array([0.9, .9, .4, .126, 0.0, 0.0108, 0.0108])  
weights_85 = np.array([0.96, .96, .4, .15, 0.0, 0.0108, 0.0108])  
weights_87 = np.array([1, 1, .4, .13, 0.0, 0.0108, 0.0108])  
weights_88 = np.array([0.96, .96, .4, .13, 0.0, 0.0108, 0.0108])  

weights_58 = np.array([0.9, .9, .4, .15, 0.0, 0.01, 0.01])   # best so far
weights_81 = np.array([0.96, .96, .45, .126, 0.0, 0.0108, 0.0108])  
weights_83 = np.array([0.96, .96, .4, .126, 0.0, 0.0108, 0.0108])  
weights_89 = np.array([0.96, .96, .4, .126, 0.0, 0.008, 0.008])  



weights_arr = np.vstack([weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, weights_7, weights_8, weights_9, weights_10, weights_11, weights_12, weights_13, weights_14,  weights_15,  weights_16,  weights_17,  weights_18,  weights_19,  weights_20,  weights_21,  weights_22,  weights_23,  weights_24,  weights_25,  weights_26,  weights_27,  weights_28,  weights_29,  weights_30,  weights_31,  weights_32,  weights_33,  weights_34,  weights_35,  weights_36,  weights_37,  weights_38,  weights_39,  weights_40,  weights_41,  weights_42,  weights_43,  weights_44,  weights_45,  weights_46,  weights_47,  weights_48,  weights_49,  weights_50,  weights_51,  weights_52,  weights_53,  weights_54,  weights_55,  weights_56,  weights_57,  weights_58,  weights_59,  weights_60,  weights_61,  weights_62,  weights_63,  weights_64,  weights_65,  weights_66,  weights_67,  weights_68,  weights_69,  weights_70,  weights_71,  weights_72,  weights_73,  weights_74,  weights_75,  weights_76,  weights_77,  weights_78,  weights_79,  weights_80,  weights_81,  weights_82,  weights_83,  weights_84,  weights_85,  weights_86,  weights_87,  weights_88,  weights_89])

In [5]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import DBSCAN

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.5], eps=0.003, dz0 = 0, stepdz = 1.5e-7, stepeps = 0.001205, num_loops=235, final_cluster=1, weight=83, weight_arr=weights_arr, max_size=16, step_z=0.000010, size_incr=0.95, min_points=1, max_cluster_size=21, final_cluster_size=2, extension_loops=20, ext_threshold=0.01, threshold_delta=1.05, z_rounds=1, loop_mod=2):
        self.max_cluster_size = max_cluster_size
        self.rz_scales=rz_scales
        self.epsilon = eps
        self.dz0 = dz0
        self.stepdz = stepdz
        self.stepeps = stepeps / num_loops
        self.num_loops = num_loops
        self.final_cluster = final_cluster
        self.weight_arr = weight_arr
        self.weights = weight
        self.max_size = max_size
        self.step_z = step_z
        if size_incr != 0:
            self.size_incr = (self.max_cluster_size - max_size) / (num_loops * size_incr)
        else:
            self.size_incr = 0
            
        self.min_points = min_points
        self.final_cluster_size = final_cluster_size
        self.extension_loops = extension_loops
        self.ext_threshold=ext_threshold
        self.threshold_delta=threshold_delta
        self.z_rounds = z_rounds
        self.loop_mod = loop_mod
        
    # remove outliers
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 5
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    
    # not sure what this function does?
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    # standard scale our data
    def _preprocess(self, hits):
        # old preprocess, may work better?
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    
    def preprocess_outliers(self, hits):
        hits2 = hits.copy()
    
        x,y,z = hits[['x', 'y', 'z']].values.T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        c = np.cos(a)
        s = np.sin(a)
        x2 = x / (r * 1000)
        y2 = y / (r * 1000)
        z2 = z / (r * 1000)
        
        hits2['c'] = c
        hits2['s'] = s
        hits2['r'] = r
        hits2['x2'] = x2
        hits2['y2'] = y2
        hits2['z2'] = z2

        return hits2
    
    def assign_outliers(self, hits):
        # add the clusters to our hits
        hits['track_id'] = self.clusters
        
        # get the size of each cluster
        hits['N1'] = hits.groupby('track_id')['track_id'].transform('count')
        
        # if the cluster has size 1 or is 0
        outliers_mask = ( (hits['N1'] == 1) | (hits['track_id'] == 0) )
        outliers = hits[outliers_mask]
        
        # mask for even tracks
        even_tracks_mask = ((hits['N1'] % 2) == 0)
        even_tracks = hits[even_tracks_mask]
        
        # make sure we have outliers and even tracks to use
        if (len(outliers) > 0) and (len(even_tracks) > 0):
            print("Event:", self.event_id, "assigning outliers")
            
            # preprocess the even tracks and fit nearest neighbors to it
            X = self.preprocess_outliers(even_tracks)
            outliers_proc = self.preprocess_outliers(outliers)

            nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(X[['c', 's', 'r']])
            indices = nbrs.kneighbors(outliers_proc[['c', 's', 'r']], n_neighbors=1, return_distance=False)
            outliers['nbr'] = indices

            nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(X)

            # assign the outliers to the track of the nearest neighbor
            for index, row in outliers.iterrows():
                outliers.loc[index, 'track_id'] = X.iloc[int(row['nbr'])]['track_id']

            # merge the outliers back into hits
            hits.loc[outliers_mask, 'track_id'] = outliers['track_id']
        
        self.clusters = hits['track_id']
        
        return self.clusters

    def _init(self,dfh):
        dfh['s1'] = dfh.hit_id
        dfh['N1'] =1
        dfh['e1'] = self.epsilon
        dfh['group_locked'] = 0
        
        mm = 1
        dz0 = self.dz0
        dfh['stepped_z'] = dfh.z
        
        z_shifts = [0, 2, 3, -2, -3]
            
        cx = self.weight_arr[self.weights]
        start_loop = self.num_loops // 5
        num_loops = start_loop + (self.num_loops // 2)
        
        for j, z_shift in enumerate(z_shifts[:self.z_rounds]):
            dfh['stepped_z'] = dfh.z - z_shift
            
            if j == 0:
                num_loops = self.num_loops
                start_loop = 0
            else:
                start_loop = self.num_loops // np.max([5 - j, 2])
                num_loops = self.num_loops - start_loop
                
            for ii in range(start_loop, num_loops):
                max_size = np.max([self.max_size + (ii * self.size_incr), self.max_cluster_size])

                dfh['r'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2+dfh['stepped_z'].values**2)
                dfh['rt'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2)
                dfh['a0'] = np.arctan2(dfh['y'].values,dfh['x'].values)
                dfh['z1'] = dfh['stepped_z'].values/dfh['rt'].values

                dfh['z2'] = dfh['stepped_z']/dfh['r']
                dfh['x2'] = dfh['x'].values/dfh['r'].values
                dfh['y2'] = dfh['y'].values/dfh['r'].values

                dfh['z3'] = np.arctan2(dfh['z'].values,dfh['rt'].values)
                mm = mm*(-1)  

                z_step = mm * self.step_z * ii
                dz = mm*((ii*self.stepdz))

                dfh['stepped_z'] = dfh['z'] + z_step    

                dfh['a1'] = dfh['a0']+mm*(dfh['rt']+(dz)*dfh['rt']**2)/1000*(ii/self.loop_mod)/180*math.pi
                dfh['sina1'] = np.sin(dfh['a1'].values)
                dfh['cosa1'] = np.cos(dfh['a1'].values)
                ss = StandardScaler()
                dfs = ss.fit_transform(dfh[['sina1','cosa1','z1','z2', 'rt', 'x2', 'y2']].values)
                dfs = np.multiply(dfs, cx)

                clusters=DBSCAN(eps=self.epsilon+(ii*self.stepeps),min_samples=self.min_points,metric='euclidean',n_jobs=1).fit(dfs).labels_ 

                if ii==0:
                    dfh['s1'] = clusters
                    dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')

                # else update our hits conditionally, if it's a better fit
                else:
                    # put our new clusters to another feature
                    dfh['s2'] = clusters
                    dfh['e2'] = self.epsilon+(ii*self.stepeps)

                    # get the count of those clusters
                    dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                    maxs1 = dfh['s1'].max()

                    # if our new clusters are bigger, but less than our max size, use the new ones instead
                    cond = np.where((dfh['N2'].values > dfh['N1'].values) & (dfh['N2'].values < max_size))

                    s1 = dfh['s1'].values
                    s1[cond] = dfh['s2'].values[cond]+maxs1

                    # write the new clusters back to our dataframe
                    dfh['s1'] = s1
                    dfh['s1'] = dfh['s1'].astype('int64')
                    dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            
        # for debugging
        # return dfh
        
        # return our clusters
        return dfh['s1'].values    
    
    def predict(self, hits, event_id):    
        original_hits = hits.copy()
        self.event_id = event_id
        
        # init our clusters
        self.clusters = self._init(hits) 
        
        # if we have unassigned points assign them with HDBSCAN
        mask = self.clusters == 0
        if (np.sum(mask) > 0) & (self.final_cluster >= 1):
            # preprocess our data
            X = self._preprocess(hits) 

            # create our last clusterer
            cl = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=self.final_cluster_size, metric='braycurtis', cluster_selection_method='leaf', algorithm='best', leaf_size=20)

            # labels = unique clusters
            labels = np.unique(self.clusters)

            # remove outliers
            self._eliminate_outliers(labels,X)
            mask = self.clusters == 0
            
            n_labels = 0
            # assign unassigned points with HDBSCANS
            max_len = np.max(self.clusters)
            
            if np.sum(mask) > 0:
                try:
                    self.clusters[mask] = cl.fit_predict(X[mask])+max_len
                except:
                    print("Error with HDBSCAN")
                
        # if we still have unassigned points assign them to their nearest neighbor
        if self.final_cluster >= 2:
            self.clusters = self.assign_outliers(hits)
            one_submission['track_id'] = self.clusters
        
        # create a submission
        one_submission = create_one_event_submission(event_id, original_hits, self.clusters)
        
        threshold = self.ext_threshold
        
        # extend the submission
        for i in range(self.extension_loops): 
            one_submission = extend(one_submission, original_hits, threshold=threshold)
            
            # decay the threshold
            threshold = threshold * self.threshold_delta
        
        self.clusters = one_submission['track_id']
        
        return one_submission

In [6]:
results_df = pd.read_pickle("gs_results.pkl")
# results_df

In [7]:
# defaults: 
# eps=0.0035, dz0 = -0.00070, stepdz = 0.00001, stepeps = 0.000005
# 0.0035, 0.0005, 5e-05, 1e-09, 150, True, 10, True

# create our params to iterate over
eps_vals = [ 0.00285 ] 
stepdz_vals = [ 1.5e-7 ] 
stepeps_vals = [ 0.001205 ] # 
loop_vals = [ 420 ] # 225
weight_vals = [ 83 ] 
step_z_vals = [ 0 ] # , 1.5e-5
size_vals = [ 16, ]
max_size_vals = [ 22, 21 ]
size_inc_vals = [ 0.95 ]
final_cluster_size_vals = [ 2 ]
final_cluster = [ 1  ] # unused
ext_delta_vals = [ 1.05 ]
z_round_vals = [ 2 ]
loop_mod_vals = [ 4 ]

dz0_vals = [ 0 ] # unused
min_size_vals = [ 1 ] # unused
min_size_change_vals = [ 0 ] # unused
dz_incr = [ True ] # unused

track_ext_round_vals = [ 8 ]

step = "eps"

foo = product(eps_vals, dz0_vals, stepdz_vals, stepeps_vals, loop_vals, final_cluster, weight_vals, dz_incr, size_vals, step_z_vals, size_inc_vals, max_size_vals, final_cluster_size_vals, track_ext_round_vals, ext_delta_vals, z_round_vals, loop_mod_vals)

offset = np.max(results_df.index.values) + 1

iter_list = []
for i, item in enumerate(foo):
    iter_list.append([i + offset, item])
    
print("Length:", len(iter_list))    

Length: 2


In [12]:
def grid_search_loop(params):
    counter = params[0]
    eps, dz0, stepdz, stepeps, loops, final_cluster, weights, dz_incr, max_size, step_z, size_incr, max_cluster_size, final_cluster_size, track_ext_rounds, ext_delta, z_rounds, loop_mod = params[1]
    step = "eps"
    dataset_scores = []
    print(" - ", counter, "Eps:", eps, "stepeps:", stepeps, "loops:", loops, "weight:", weights, "max size:", max_size, "step z:", step_z, "max:", max_cluster_size, "ext rounds:", track_ext_rounds)
    
    for event_id, hits, cells, particles, truth in load_dataset("./data/train_1", skip=50, nevents=5):
        # Track pattern recognition
        model = Clusterer(eps=eps, dz0=dz0, stepdz=stepdz, stepeps=stepeps, num_loops=loops, final_cluster=final_cluster, weight=weights, weight_arr=weights_arr, max_size=max_size, step_z=step_z, size_incr=size_incr, max_cluster_size=max_cluster_size, final_cluster_size=final_cluster_size, extension_loops=track_ext_rounds, threshold_delta=ext_delta, z_rounds= z_rounds, loop_mod=loop_mod)
        one_submission = model.predict(hits.copy(), event_id)

        # Score for the event
        score = score_event(truth, one_submission)
        dataset_scores.append(score)
        print(event_id, params[1], "score:", score)
        
    mean_score = np.mean(dataset_scores)
    
    # create and return our results
    result = [counter, [eps, dz0, stepdz, stepeps, loops, step, weights, final_cluster, max_size, step_z, size_incr, max_cluster_size, final_cluster_size, track_ext_rounds, ext_delta, z_rounds, loop_mod, mean_score]]
    print(result)
    
#     return one_submission
    return result

def grid_search(iter_list, results_df=None, start=0, end=5):
    pool = Pool(processes=15)
    results = pool.map(grid_search_loop, iter_list[start:end])
    pool.close()
    
    return results

In [13]:
results = grid_search(iter_list, results_df, start=0, end=16)

for item in results:
    results_df.loc[item[0]] = item[1]
    
results_df.sort_values("acc", ascending=False).to_pickle("gs_results.pkl")
results_df

 -  1656 Eps: 0.00285 stepeps: 0.001205 loops: 420 weight: 83 max size: 16 step z: 0 max: 22 ext rounds: 8
 -  1657 Eps: 0.00285 stepeps: 0.001205 loops: 420 weight: 83 max size: 16 step z: 0 max: 21 ext rounds: 8
 -  1658 Eps: 0.00285 stepeps: 0.001205 loops: 350 weight: 83 max size: 16 step z: 0 max: 22 ext rounds: 8
 -  1660 Eps: 0.00285 stepeps: 0.001205 loops: 335 weight: 83 max size: 16 step z: 0 max: 22 ext rounds: 8
 -  1659 Eps: 0.00285 stepeps: 0.001205 loops: 335 weight: 83 max size: 16 step z: 0 max: 21 ext rounds: 8
1050 (0.00285, 0, 1.5e-07, 0.001205, 335, 1, 83, True, 16, 0, 0.95, 22, 2, 8, 1.05, 2, 3) score: 0.5772603882085939
1050 (0.00285, 0, 1.5e-07, 0.001205, 335, 1, 83, True, 16, 0, 0.95, 21, 2, 8, 1.05, 2, 3) score: 0.5774700708389706
1050 (0.00285, 0, 1.5e-07, 0.001205, 350, 1, 83, True, 16, 0, 0.95, 22, 2, 8, 1.05, 2, 3) score: 0.5809697746937537
1050 (0.00285, 0, 1.5e-07, 0.001205, 420, 1, 83, True, 16, 0, 0.95, 22, 2, 8, 1.05, 2, 4) score: 0.5790047932013369
1

,eps,dz0,stepdz,stepeps,loops,step,weights,fl_clust,max_size,step_z,size_incr,cluster_max,fl_cl_size,tr_ext_rnds,d_ext,z_rnds,loop_mod,acc
1654,0.00285,0.0000,1.500000e-07,1.205000e-03,350.0,eps,83,1,16,0.000000,0.95,22,2,8,1.05,2,3,0.578975
1655,0.00285,0.0000,1.500000e-07,1.205000e-03,350.0,eps,83,1,16,0.000000,0.95,21,2,8,1.05,2,3,0.578963
1635,0.00285,0.0000,1.500000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,0.95,22,2,8,1.05,2,2,0.578120
1631,0.00285,0.0000,1.500000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,1.00,22,2,8,1.05,2,2,0.578120
1633,0.00285,0.0000,1.500000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,0.95,21,2,8,1.05,2,2,0.578107
1629,0.00285,0.0000,1.500000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,1.00,21,2,8,1.05,2,2,0.578107
1625,0.00285,0.0000,1.000000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,0.95,21,2,8,1.05,2,2,0.577961
1621,0.00285,0.0000,1.000000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,1.00,21,2,8,1.05,2,2,0.577961
1630,0.00285,0.0000,1.500000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,1.00,21,2,8,1.05,3,2,0.577924
1634,0.00285,0.0000,1.500000e-07,1.205000e-03,235.0,eps,83,1,16,0.000000,0.95,21,2,8,1.05,3,2,0.577924


In [ ]:
results_df.sort_values("acc", ascending=False)

In [ ]:
# get one submission to play with
submission = grid_search_loop(iter_list[0])

In [15]:
results_df.sort_values("acc", ascending=False).to_pickle("gs_results.pkl")

In [18]:
results_df.loc[(results_df.cluster_max == 22) & (results_df.z_rnds == 2)].sort_values("acc", ascending=False)

,eps,dz0,stepdz,stepeps,loops,step,weights,fl_clust,max_size,step_z,size_incr,cluster_max,fl_cl_size,tr_ext_rnds,d_ext,z_rnds,acc
1631,0.00285,0.0,1.500000e-07,0.001205,235.0,eps,83,1,16,0.0,1.00,22,2,8,1.05,2,0.578120
1635,0.00285,0.0,1.500000e-07,0.001205,235.0,eps,83,1,16,0.0,0.95,22,2,8,1.05,2,0.578120
1623,0.00285,0.0,1.000000e-07,0.001205,235.0,eps,83,1,16,0.0,1.00,22,2,8,1.05,2,0.577864
1627,0.00285,0.0,1.000000e-07,0.001205,235.0,eps,83,1,16,0.0,0.95,22,2,8,1.05,2,0.577864
1619,0.00285,0.0,1.500000e-07,0.001205,235.0,eps,83,1,16,0.0,0.95,22,2,8,1.05,2,0.577244
1644,0.00285,0.0,1.500000e-07,0.001205,235.0,eps,83,1,16,0.0,0.95,22,2,8,1.05,2,0.577060
1613,0.00285,0.0,1.500000e-07,0.001005,235.0,eps,83,1,16,0.0,0.95,22,2,8,1.05,2,0.576478
1638,0.00285,0.0,2.000000e-07,0.001205,235.0,eps,83,1,16,0.0,0.95,22,2,8,1.05,2,0.576468


## Scratch

In [95]:
def preprocess_outliers(hits):
    hits2 = hits.copy()
    
    x,y,z = hits[['x', 'y', 'z']].values.T
    r  = (x**2 + y**2)**0.5
    r  = r/1000
    a  = np.arctan2(y,x)
    c = np.cos(a)
    s = np.sin(a)

    hits2['c'] = c
    hits2['s'] = s
    hits2['r'] = r
    
    return hits2
    
def assign_outliers(hits):
    # add the clusters to our hits
#     hits['track_id'] = self.clusters

    # get the size of each cluster
    hits['N1'] = hits.groupby('track_id')['track_id'].transform('count')

    # if the cluster has size 1 or is 0
    mask = (hits['N1'] == 1 | hits['track_id'] == 0)
    outliers = hits[mask]

    # mask for even tracks
    even_tracks_mask = ((hits['N1'] % 2) == 0)
    even_tracks = hits[even_tracks_mask]

    # preprocess the even tracks and fit nearest neighbors to it
    X = preprocess_outliers(even_tracks)
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(X)

    # preprocess the outliers and get the nearest neighbor for each
    outliers = preprocess_outliers(outliers)
    indices = nbrs.kneighbors(outliers, n_neighbors=1, return_distance=False)
    outliers['nbr'] = indices
    
    # assign the outliers to the track of the nearest neighbor
    for row in outliers.iterrows():
        neighbor = X.iloc[row['nbr']]
        

    return hits

In [96]:
for event_id, hits, cells, particles, truth in load_dataset("./data/train_1", skip=50, nevents=1):
    hits['track_id'] = submission['track_id']

In [97]:
hits['N1'] = hits.groupby('track_id')['track_id'].transform('count')

mask = ( (hits['N1'] == 1) | (hits['track_id'] == 0) )
outliers = hits[mask]

# mask for even tracks
even_tracks_mask = ((hits['N1'] % 2) == 0)
even_tracks = hits[even_tracks_mask]

In [98]:
X = preprocess_outliers(even_tracks)
outliers_proc = preprocess_outliers(outliers)

nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(X[['c', 's', 'r']])
indices = nbrs.kneighbors(outliers_proc[['c', 's', 'r']], n_neighbors=1, return_distance=False)
outliers['nbr'] = indices

/home/skooch/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [99]:
outliers

,hit_id,x,y,z,volume_id,layer_id,module_id,track_id,N1,nbr
32648,32649,62.141201,98.091499,-128.850006,8,6,295,1949622,1,17112
33495,33496,-27.344801,114.413002,-51.299801,8,6,354,18409903,1,17360
33551,33552,-67.969498,95.379700,-17.507700,8,6,357,11685994,1,16835
34974,34975,-110.622002,36.811901,115.222000,8,6,466,9636902,1,2567
41705,41706,-60.984100,-160.223999,369.039001,8,8,951,18521710,1,20502
42091,42092,171.774002,6.021760,468.938995,8,8,1054,18521332,1,18734
58239,58240,-560.581970,-30.300100,-2948.500000,12,2,3,18523761,1,30068
59251,59252,-526.934998,441.579010,-2945.500000,12,2,152,18521978,1,30562
61271,61272,292.415009,534.953003,-2148.500000,12,6,116,3937267,1,31021
61513,61514,-324.071991,266.165009,-2151.500000,12,6,151,18523119,1,46446


In [106]:
for index, row in outliers.iterrows():
    print(X.iloc[int(row['nbr'])]['track_id'])
    outliers.loc[index, 'track_id'] = X.iloc[int(row['nbr'])]['track_id']

18524844.0
8719220.0
1575617.0
11582980.0
2789076.0
18520929.0
6648030.0
18521324.0
18522229.0
1347411.0


/home/skooch/.local/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


165095.0
5919807.0
18523733.0
1864044.0
18525541.0
18526747.0
2552722.0
18521714.0
18526105.0
6737515.0
18525952.0
18525680.0
3946074.0
18526114.0
18524018.0
4025041.0
3906428.0
7227919.0
18524913.0
18525458.0
18525977.0
5489690.0
18522829.0
18524172.0
18523786.0


In [108]:
hits.loc[mask, 'track_id'] = outliers['track_id']

In [109]:
outliers

,hit_id,x,y,z,volume_id,layer_id,module_id,track_id,N1,nbr
32648,32649,62.141201,98.091499,-128.850006,8,6,295,18524844.0,1,17112
33495,33496,-27.344801,114.413002,-51.299801,8,6,354,8719220.0,1,17360
33551,33552,-67.969498,95.379700,-17.507700,8,6,357,1575617.0,1,16835
34974,34975,-110.622002,36.811901,115.222000,8,6,466,11582980.0,1,2567
41705,41706,-60.984100,-160.223999,369.039001,8,8,951,2789076.0,1,20502
42091,42092,171.774002,6.021760,468.938995,8,8,1054,18520929.0,1,18734
58239,58240,-560.581970,-30.300100,-2948.500000,12,2,3,6648030.0,1,30068
59251,59252,-526.934998,441.579010,-2945.500000,12,2,152,18521324.0,1,30562
61271,61272,292.415009,534.953003,-2148.500000,12,6,116,18522229.0,1,31021
61513,61514,-324.071991,266.165009,-2151.500000,12,6,151,1347411.0,1,46446


In [110]:
hits['N1'] = hits.groupby('track_id')['track_id'].transform('count')

mask = ( (hits['N1'] == 1) | (hits['track_id'] == 0) )

hits[mask]

,hit_id,x,y,z,volume_id,layer_id,module_id,track_id,N1


112098

### EDA

In [66]:
for event_id, hits, cells, particles, truth in load_dataset("./data/train_1", skip=40, nevents=1):
    # Track pattern recognition
    model = Clusterer(eps=0.0035, dz0=0, stepdz=1.5e-7, stepeps=7e-8, num_loops=100, final_cluster=True, weight=58, weight_arr=weights_arr, max_size=20, step_z=0.000045, size_incr=0)
    labels = model._init(hits)

In [10]:
labels

,hit_id,x,y,z,volume_id,layer_id,module_id,s1,N1,stepped_z,...,z1,z2,z3,a1,x2,y2,sina1,cosa1,s2,N2
0,1,-48.674801,-5.761740,-1502.5,7,2,1,0,2,-1502.495605,...,-30.654200,-0.999468,-0.032622,-2.981392,-0.032379,-0.003833,-0.159516,-0.987195,0,2
1,2,-75.548500,-9.682310,-1502.5,7,2,1,1822912,15,-1502.495605,...,-19.726601,-0.998718,-0.050693,-2.948250,-0.050217,-0.006436,-0.192141,-0.981367,1,2
2,3,-69.968399,-5.581860,-1502.5,7,2,1,2,1,-1502.495605,...,-21.406033,-0.998911,-0.046716,-3.001281,-0.046517,-0.003711,-0.139852,-0.990172,2,1
3,4,-35.324799,-2.704890,-1502.5,7,2,1,3,1,-1502.495605,...,-42.409836,-0.999722,-0.023579,-3.034546,-0.023504,-0.001800,-0.106842,-0.994276,3,1
4,5,-99.261803,-11.197500,-1502.5,7,2,1,5588395,4,-1502.495605,...,-15.041381,-0.997797,-0.066483,-2.942832,-0.065919,-0.007436,-0.197455,-0.980312,4,1
5,6,-67.558197,-6.094340,-1502.5,7,2,1,5,14,-1502.495605,...,-22.150208,-0.998982,-0.045146,-2.992965,-0.044918,-0.004052,-0.148081,-0.988975,5,1
6,7,-63.736301,3.384740,-1502.5,7,2,1,6,11,-1502.495605,...,-23.540590,-0.999099,-0.042480,3.143731,-0.042382,0.002251,-0.002139,-0.999998,6,2
7,8,-59.426899,-12.626500,-1502.5,7,2,1,4726944,12,-1502.495605,...,-24.731169,-0.999184,-0.040435,-2.879700,-0.039520,-0.008397,-0.258909,-0.965902,7,2
8,9,-78.164299,-1.813290,-1502.5,7,2,1,8,2,-1502.495605,...,-19.217215,-0.998649,-0.052037,-3.050773,-0.051952,-0.001205,-0.090695,-0.995879,8,2
9,10,-83.351303,0.286217,-1502.5,7,2,1,3127933,16,-1502.495605,...,-18.026058,-0.998465,-0.055475,3.210259,-0.055390,0.000190,-0.068612,-0.997643,9,2


In [37]:
cond = np.where((labels['N2'].values > labels['N1'].values) & (labels['N2'].values < 20))

In [58]:
cond = np.where((labels['N2'].values > labels['N1'].values) & ( ((labels['N2'].values < 20) & ( (labels['x'] > 275) | (labels['y'] > 625))) | (labels['N2'] < 15)))

In [46]:
( ((labels['N2'].values < 20) & (labels['x'].values > 275 | labels['y'].values > 625)) | (labels['N2'].values < 15))

TypeError: ufunc 'bitwise_or' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [56]:
(labels['x'].values > 275) | (labels['y'].values > 625)

array([False, False, False, ..., False, False, False])

In [55]:
labels['y'].values > 625

array([False, False, False, ..., False, False, False])

In [26]:
labels.loc[cond]

,hit_id,x,y,z,volume_id,layer_id,module_id,s1,N1,stepped_z,...,z1,z2,z3,a1,x2,y2,sina1,cosa1,s2,N2
319,320,-60.398602,-127.692001,-1502.0,7,2,21,8348561,1,-1501.995605,...,-10.633213,-0.995607,-0.094045,-1.890321,-0.040035,-0.084641,-0.949385,-0.314115,241,2
343,344,-23.916300,-97.369598,-1502.5,7,2,23,8348576,2,-1502.495605,...,-14.985513,-0.997781,-0.066731,-1.724901,-0.015882,-0.064661,-0.988149,-0.153496,256,13
363,364,-32.573200,-138.384003,-1502.0,7,2,24,8348592,2,-1501.995605,...,-10.565152,-0.995550,-0.094651,-1.678889,-0.021590,-0.091723,-0.994164,-0.107882,272,13
370,371,-35.410400,-138.871994,-1502.0,7,2,24,8348597,2,-1501.995605,...,-10.480405,-0.995479,-0.095416,-1.696383,-0.023469,-0.092040,-0.992124,-0.125257,277,7
391,392,-11.331600,-113.625000,-1498.0,7,2,26,8348613,6,-1497.995605,...,-13.118681,-0.997107,-0.076227,-1.571377,-0.007543,-0.075631,-1.000000,-0.000580,293,16
393,394,-11.398400,-113.919998,-1502.0,7,2,27,8348613,6,-1501.995605,...,-13.119224,-0.997108,-0.076224,-1.571441,-0.007567,-0.075626,-1.000000,-0.000645,293,16
492,493,57.278702,-166.098007,-1502.0,7,2,34,8348688,1,-1501.995605,...,-8.548839,-0.993228,-0.116975,-1.086531,0.037877,-0.109835,-0.885017,0.465559,368,3
649,650,134.445007,-66.787003,-1502.0,7,2,46,8348804,7,-1501.995605,...,-10.005369,-0.995042,-0.099946,-0.331070,0.089067,-0.044245,-0.325055,0.945695,484,8
667,668,58.875198,-23.493900,-1498.0,7,2,47,8348816,1,-1497.995605,...,-23.631674,-0.999106,-0.042316,-0.324867,0.039267,-0.015669,-0.319183,0.947693,496,11
688,689,134.128006,-66.574799,-1498.0,7,2,48,8348804,7,-1497.995605,...,-10.003934,-0.995041,-0.099961,-0.331070,0.089094,-0.044222,-0.325056,0.945695,484,8
